# Aim 3: find out the characteristics of more susceptible group

1. divide patients into two groups: who is hyperchloremic between 24 and 48h
2. look at the variables from the first 24h 
3. univariate analysis

In [81]:
import numpy as np
import pandas as pd
import datetime
import statsmodels.api as sm

from IPython.display import display
from scipy.stats import chi2_contingency, kruskal
from tableone import TableOne

pd.options.display.precision = 3

## Load useful DataFrames

In [97]:
# Target Group: adult patients' first icu visit
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)

# Create ICU Id list
icu = []
for icuid in df['icustay_id']:
    icu.append(icuid)

# Max chloride
df = pd.read_csv('chloride.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.set_index(['subject_id','hadm_id'])
df = df.filter(['icustay_id','icu_day','chloride_max'])
df = df.dropna(subset = ['chloride_max'])

# Chloride >= 110 in the second day
df_chl = df.query('icu_day == 2')
df_chl['chl_110'] = (df_chl['chloride_max'] >= 110).astype(int)

# Demographic
df = pd.read_csv('adm_demographics.csv')
df = df.set_index(['subject_id', 'hadm_id'])
df_demo = df.filter(['insurance','ethnicity','age','gender'])

# Weight
df = pd.read_csv('weight.csv')
df = df[df['icustay_id'].isin(icu)]
df_wt = df.filter(['icustay_id','day','weight'])

# GCS(Glasgow coma scale)
df = pd.read_csv('gcs_pan.csv')
df = df[df['icustay_id'].isin(icu)]
df_gcs = df.filter(['icustay_id','day','mingcs'])

# First-day Vitals(Merge with GCS and Weight)
df = pd.read_csv('vitals_pan.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.set_index(['subject_id','hadm_id','icustay_id'])
df_vitals = df.filter(['day', 'heartrate_max','sysbp_min','diasbp_min','resprate_max','spo2_min','tempc_max'])
df_vitals = df_vitals.reset_index()
df_vitals = df_vitals.merge(df_wt, on = ['icustay_id','day']).merge(df_gcs, on = ['icustay_id','day'])
df_vitals = df_vitals.query('(day == 1) & (0 < heartrate_max <= 250) &(40<sysbp_min <190) &(20<diasbp_min<150) &(6<resprate_max<80) &(0<spo2_min<100)&(30<tempc_max<44)&(30<weight<500)&(3<mingcs<15)'
                           )

# Medication from prescription table
df = pd.read_csv('medication.csv')
df_med = df.drop(df.columns[[0,3]], axis=1)
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)
df = df_med.merge(df, on = ['subject_id','hadm_id'])
df['taken'] = ((pd.to_datetime(df['intime']) <= pd.to_datetime(df['startdate'])) & 
               ((pd.to_datetime(df['startdate']) <= (pd.to_datetime(df['intime']) + datetime.timedelta(days=1)))))
df = df.filter(['subject_id','hadm_id','icustay_id','drug','taken'])
df_med = df.set_index(['subject_id','hadm_id','icustay_id'])
df_med = df_med.groupby(['subject_id','hadm_id','icustay_id','drug']).any()
df_med = df_med['taken'].unstack()
df_med = (df_med * 1).fillna(0)

# Intake and output
df = pd.read_csv('fluid_io.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.query('icu_day == 1')
df_fluid = df.set_index(['subject_id','hadm_id','icustay_id'])
df_fluid.head()

# Diagnosis Code(group by elixhauser)
df = pd.read_csv('elixhauser.csv')
df_icd = df.drop(df.columns[[0]], axis=1)

# Labs
df = pd.read_csv('labs1.csv')
df = df.drop(df.columns[0], axis=1)
df = df[df['icustay_id'].isin(icu)]
df_labs = df.query('day == 1 & 0.1<lactate_max<40 & 90<sodium_max<190 & 1.5<potassium_max<15 & 1<albumin_max<6.5 & 5<bicarbonate_min<65 & 0.1<creatinine_max<47 & 66<chloride_max<170 & 18<glucose_max<2500 & 2<hemoglobin_min<21 & 5<platelet_min<1600  & 0.5<inr_max<20 & 1<bun_max<280 & 0.1<wbc_max<680 & 0.1<bilirubin_min<80 & 1.5<calcium_total_min<18 & 0.2<calcium_ionized_min<2.5 & 2<ast_sgot_max<25000 & 3<amylase_max<25000 &  1<lipase_max<25000 & 0.1<c_reactive_protein_max<300')


/Users/panyiheng/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Summary statistics

## _Table 1(a). Demographics_

In [83]:
df_chl = df_chl.reset_index()
df_chl = df_chl.set_index(['subject_id','hadm_id'])
df_combined = df_demo.join(df_chl, how='inner')
columns = ['age', 'gender', 'ethnicity']
categorical = ['gender', 'ethnicity']
groupby = 'chl_110'
display(TableOne(df_combined, columns, categorical, groupby,
                 labels={'age': 'Age', 'gender': 'Gender', 'ethnicity': 'Ethnicity',
                         'chl_110': 'Hyperchloremia (>=110)'},
                 pval=True, isnull=False, label_suffix=True))

Grouped by Hyperchloremia (>=110)                                        
                                                          0            1    pval              ptest
variable         level                                                                             
n                                                     24452         7293                           
Age, mean (SD)                                  64.4 (16.7)  66.0 (17.7)  <0.001  Two Sample T-test
Gender, n (%)    F                             10415 (42.6)  3646 (50.0)  <0.001        Chi-squared
                 M                             14037 (57.4)  3647 (50.0)                           
Ethnicity, n (%) ASIAN                            496 (2.0)    226 (3.1)  <0.001        Chi-squared
                 BLACK                           2254 (9.2)   733 (10.1)                           
                 HISPANIC                         767 (3.1)    240 (3.3)                           
                 OTHER                          3314 (13.6)  1020 (14.0)                           
                 WHITE                         17621 (72.1)  5074 (69.6)                           
[1] Warning, Hartigan's Dip Test reports possible multimodal distributions for: age.
[2] Warning, test for normality reports non-normal distributions for: age.

## _Table 1(b). Vitals_

In [84]:
df_chl = df_chl.reset_index()
df_chl = df_chl.set_index(['subject_id','hadm_id','icustay_id'])
df_vitals = df_vitals.reset_index()
df_vitals = df_vitals.set_index(['subject_id','hadm_id','icustay_id'])
df_combined = df_vitals.join(df_chl, how='inner')

columns = ['heartrate_max','sysbp_min','diasbp_min','resprate_max','spo2_min','tempc_max','weight','mingcs']
groupby = 'chl_110'
display(TableOne(df_combined, columns,categorical = [], groupby = 'chl_110',
                 labels={
                         'chl_110': 'Hyperchloremia (>=110)'},
                 pval=True, isnull=False, label_suffix=True))

Grouped by Hyperchloremia (>=110)                                         
                                                               0             1    pval              ptest
variable                 level                                                                           
n                                                           3927          1237                           
heartrate_max, mean (SD)                            103.8 (19.9)  107.2 (21.6)  <0.001  Two Sample T-test
sysbp_min, mean (SD)                                 93.9 (17.4)   90.5 (17.8)  <0.001  Two Sample T-test
diasbp_min, mean (SD)                                44.5 (11.3)   42.2 (11.1)  <0.001  Two Sample T-test
resprate_max, mean (SD)                               27.5 (6.3)    28.2 (6.6)  <0.001  Two Sample T-test
spo2_min, mean (SD)                                   91.2 (7.8)    91.8 (7.3)   0.018  Two Sample T-test
tempc_max, mean (SD)                                  37.5 (0.8)    37.5 (0.8)   0.492  Two Sample T-test
weight, mean (SD)                                    80.4 (23.7)   73.7 (18.9)  <0.001  Two Sample T-test
mingcs, mean (SD)                                     12.1 (2.6)    11.2 (2.9)  <0.001  Two Sample T-test
[1] Warning, Hartigan's Dip Test reports possible multimodal distributions for: diasbp_min, heartrate_max, mingcs, resprate_max, spo2_min, sysbp_min, tempc_max.
[2] Warning, Tukey test indicates far outliers in: heartrate_max, resprate_max, spo2_min, tempc_max, weight.
[3] Warning, test for normality reports non-normal distributions for: diasbp_min, heartrate_max, mingcs, resprate_max, spo2_min, sysbp_min, tempc_max, weight.

## _Table 1(c). Medication_ 

In [85]:
df_chl = df_chl.reset_index()
df_chl = df_chl.set_index(['subject_id','hadm_id','icustay_id'])
df_combined = df_med.join(df_chl, how='inner')
columns = ['dopamine','epinephrine','norepinephrine','vasopressin']
categorical = ['dopamine','epinephrine','norepinephrine','vasopressin']
groupby = 'chl_110'
display(TableOne(df_combined, columns,categorical, groupby = 'chl_110',
                                  labels={
                         'chl_110': 'Hyperchloremia (>=110)'},
                 pval=True, isnull=False, label_suffix=True))

Grouped by Hyperchloremia (>=110)                                  
                                                            0            1    pval        ptest
variable              level                                                                    
n                                                        5439         2340                     
dopamine, n (%)       0                           4976 (91.5)  2166 (92.6)   0.123  Chi-squared
                      1                             463 (8.5)    174 (7.4)                     
epinephrine, n (%)    0                           5191 (95.4)  2249 (96.1)   0.205  Chi-squared
                      1                             248 (4.6)     91 (3.9)                     
norepinephrine, n (%) 0                           4155 (76.4)  1646 (70.3)  <0.001  Chi-squared
                      1                           1284 (23.6)   694 (29.7)                     
vasopressin, n (%)    0                           4997 (91.9)  2133 (91.2)   0.313  Chi-squared
                      1                             442 (8.1)    207 (8.8)

## _Table 1(d). FLuid balance_

In [86]:
df_chl = df_chl.reset_index()
df_chl = df_chl.set_index(['subject_id','hadm_id','icustay_id'])
df_chl = df_chl.drop(df_chl.columns[[0]], axis = 1)
df_combined = df_fluid.join(df_chl, how='inner')

In [87]:
columns = ['fluid_net_input_ml']
groupby = 'chl_110'
display(TableOne(df_combined, columns,categorical = [], groupby = 'chl_110',
                 labels={
                         'chl_110': 'Hyperchloremia (>=110)'},
                 pval=True, isnull=False, label_suffix=True))

Grouped by Hyperchloremia (>=110)                                            
                                                                    0                1    pval              ptest
variable                      level                                                                              
n                                                               19464             4797                           
fluid_net_input_ml, mean (SD)                         1051.8 (1955.6)  1646.4 (1900.0)  <0.001  Two Sample T-test
[1] Warning, test for normality reports non-normal distributions for: fluid_net_input_ml.

## _Table 1(e). Diagnosis_

In [128]:
df_chl = df_chl.reset_index()
df_combined = df_chl.merge(df_icd, on = 'hadm_id')

In [129]:
columns = ['congestive_heart_failure',
       'cardiac_arrhythmias', 'valvular_disease', 'pulmonary_circulation',
       'peripheral_vascular', 'hypertension', 'paralysis',
       'other_neurological', 'chronic_pulmonary', 'diabetes_uncomplicated',
       'diabetes_complicated', 'hypothyroidism', 'renal_failure',
       'liver_disease', 'peptic_ulcer', 'aids', 'lymphoma',
       'metastatic_cancer', 'solid_tumor', 'rheumatoid_arthritis',
       'coagulopathy', 'obesity', 'weight_loss', 'fluid_electrolyte',
       'blood_loss_anemia', 'deficiency_anemias', 'alcohol_abuse',
       'drug_abuse', 'psychoses', 'depression']

categorical = ['congestive_heart_failure',
       'cardiac_arrhythmias', 'valvular_disease', 'pulmonary_circulation',
       'peripheral_vascular', 'hypertension', 'paralysis',
       'other_neurological', 'chronic_pulmonary', 'diabetes_uncomplicated',
       'diabetes_complicated', 'hypothyroidism', 'renal_failure',
       'liver_disease', 'peptic_ulcer', 'aids', 'lymphoma',
       'metastatic_cancer', 'solid_tumor', 'rheumatoid_arthritis',
       'coagulopathy', 'obesity', 'weight_loss', 'fluid_electrolyte',
       'blood_loss_anemia', 'deficiency_anemias', 'alcohol_abuse',
       'drug_abuse', 'psychoses', 'depression']
groupby = 'chl_110'
display(TableOne(df_combined, columns,categorical, groupby = 'chl_110',
                                  labels={
                         'chl_110': 'Hyperchloremia (>=110)'},
                 pval=True, isnull=False, label_suffix=True))

Grouped by Hyperchloremia (>=110)                                  
                                                                      0            1    pval        ptest
variable                        level                                                                    
n                                                                 24452         7293                     
congestive_heart_failure, n (%) 0.0                        16312 (66.8)  5303 (72.9)  <0.001  Chi-squared
                                1.0                         8092 (33.2)  1970 (27.1)                     
cardiac_arrhythmias, n (%)      0.0                        14672 (60.1)  4680 (64.3)  <0.001  Chi-squared
                                1.0                         9732 (39.9)  2593 (35.7)                     
valvular_disease, n (%)         0.0                        20989 (86.0)  6559 (90.2)  <0.001  Chi-squared
                                1.0                         3415 (14.0)    714 (9.8)                     
pulmonary_circulation, n (%)    0.0                        22381 (91.7)  6874 (94.5)  <0.001  Chi-squared
                                1.0                          2023 (8.3)    399 (5.5)                     
peripheral_vascular, n (%)      0.0                        22157 (90.8)  6706 (92.2)  <0.001  Chi-squared
                                1.0                          2247 (9.2)    567 (7.8)                     
hypertension, n (%)             0                          10940 (44.7)  3475 (47.6)  <0.001  Chi-squared
                                1                          13512 (55.3)  3818 (52.4)                     
paralysis, n (%)                0.0                        23731 (97.2)  6983 (96.0)  <0.001  Chi-squared
                                1.0                           673 (2.8)    290 (4.0)                     
other_neurological, n (%)       0.0                        21721 (89.0)  6100 (83.9)  <0.001  Chi-squared
                                1.0                         2683 (11.0)  1173 (16.1)                     
chronic_pulmonary, n (%)        0.0                        18360 (75.2)  5927 (81.5)  <0.001  Chi-squared
                                1.0                         6044 (24.8)  1346 (18.5)                     
diabetes_uncomplicated, n (%)   0                          19377 (79.2)  5896 (80.8)   0.003  Chi-squared
                                1                           5075 (20.8)  1397 (19.2)                     
diabetes_complicated, n (%)     0.0                        22505 (92.2)  6802 (93.5)  <0.001  Chi-squared
                                1.0                          1899 (7.8)    471 (6.5)                     
hypothyroidism, n (%)           0.0                        21771 (89.2)  6487 (89.2)   0.983  Chi-squared
                                1.0                         2633 (10.8)   786 (10.8)                     
renal_failure, n (%)            0.0                        20105 (82.4)  6096 (83.8)   0.005  Chi-squared
                                1.0                         4299 (17.6)  1177 (16.2)                     
liver_disease, n (%)            0.0                        21763 (89.2)  6325 (87.0)  <0.001  Chi-squared
                                1.0                         2641 (10.8)   948 (13.0)                     
peptic_ulcer, n (%)             0.0                        24193 (99.1)  7193 (98.9)   0.076  Chi-squared
                                1.0                           211 (0.9)     80 (1.1)                     
aids, n (%)                     0.0                        24251 (99.4)  7215 (99.2)   0.137  Chi-squared
                                1.0                           153 (0.6)     58 (0.8)                     
lymphoma, n (%)                 0.0                        24031 (98.5)  7170 (98.6)   0.525  Chi-squared
                                1.0                           373 (1.5)    103 (1.4)                     
metastatic

In [159]:
diag_summary = TableOne(df_combined, columns,categorical, groupby = 'chl_110',
                                  labels={
                         'chl_110': 'Hyperchloremia (>=110)'},
                 pval=True, isnull=False, label_suffix=True)
diag_summary.to_csv('diag_summary.csv')

In [161]:
diag_summary = pd.read_csv('diag_summary.csv')
diag_summary = diag_summary.drop(diag_summary.index[[0,1,2]])
diag_summary.columns = ['variable', 'level', '0', '1', 'pval', 'ptest']
diag_summary['pval'] = diag_summary['pval'].str.replace('<', '')
diag_summary.pval = diag_summary.pval.astype(float).fillna(1.0)

In [162]:
diag_sig = diag_summary.loc[diag_summary['pval'] <= 0.05]
diag_sig

,variable,level,0,1,pval,ptest
3,"congestive_heart_failure, n (%)",0.0,16312 (66.8),5303 (72.9),0.001,Chi-squared
5,"cardiac_arrhythmias, n (%)",0.0,14672 (60.1),4680 (64.3),0.001,Chi-squared
7,"valvular_disease, n (%)",0.0,20989 (86.0),6559 (90.2),0.001,Chi-squared
9,"pulmonary_circulation, n (%)",0.0,22381 (91.7),6874 (94.5),0.001,Chi-squared
11,"peripheral_vascular, n (%)",0.0,22157 (90.8),6706 (92.2),0.001,Chi-squared
13,"hypertension, n (%)",0,10940 (44.7),3475 (47.6),0.001,Chi-squared
15,"paralysis, n (%)",0.0,23731 (97.2),6983 (96.0),0.001,Chi-squared
17,"other_neurological, n (%)",0.0,21721 (89.0),6100 (83.9),0.001,Chi-squared
19,"chronic_pulmonary, n (%)",0.0,18360 (75.2),5927 (81.5),0.001,Chi-squared
21,"diabetes_uncomplicated, n (%)",0,19377 (79.2),5896 (80.8),0.003,Chi-squared


## _Table 1(e). Labs_

In [90]:
df_chl = df_chl.reset_index()
df_chl = df_chl.set_index(['subject_id','hadm_id','icustay_id'])
df_labs = df_labs.set_index(['subject_id','hadm_id','icustay_id'])
df_labs = df_labs.drop(df_labs.columns[[0]], axis=1)
df_chl_110 = df_chl.filter(['chl_110'])
df_combined = df_labs.join(df_chl_110, how='inner')

In [91]:
columns = ['lactate_max','sodium_max','potassium_max','albumin_max','bicarbonate_min','creatinine_max','chloride_max','glucose_max','hemoglobin_min','platelet_min','potassium_min','inr_max','bun_max','wbc_max','bilirubin_min','calcium_total_min','calcium_ionized_min','ast_sgot_max','amylase_max','lipase_max','c_reactive_protein_max']
categorical = []
groupby = 'chl_110'
display(TableOne(df_combined, columns,categorical, groupby = 'chl_110',
                                  labels={
                         'chl_110': 'Hyperchloremia (>=110)'},
                 pval=True, isnull=False, label_suffix=True))

/Users/panyiheng/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:1394: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  "anyway, n=%i" % int(n))


Grouped by Hyperchloremia (>=110)                                          
                                                                        0              1    pval              ptest
variable                          level                                                                            
n                                                                      11             10                           
lactate_max, mean (SD)                                          2.2 (1.5)      2.1 (1.2)   0.802  Two Sample T-test
sodium_max, mean (SD)                                         138.5 (3.2)    142.3 (3.7)   0.023  Two Sample T-test
potassium_max, mean (SD)                                        4.6 (0.6)      4.5 (0.4)   0.682  Two Sample T-test
albumin_max, mean (SD)                                          2.8 (0.6)      2.7 (1.0)   0.808  Two Sample T-test
bicarbonate_min, mean (SD)                                     20.9 (4.1)     15.9 (3.1)   0.005  Two Sample T-test
creatinine_max, mean (SD)                                       2.4 (3.5)      1.9 (1.4)   0.660  Two Sample T-test
chloride_max, mean (SD)                                       105.7 (3.6)    116.7 (3.6)  <0.001  Two Sample T-test
glucose_max, mean (SD)                                       196.2 (86.8)  231.4 (126.4)   0.472  Two Sample T-test
hemoglobin_min, mean (SD)                                       9.8 (1.6)      9.2 (2.0)   0.442  Two Sample T-test
platelet_min, mean (SD)                                      169.1 (70.9)  198.0 (203.8)   0.678  Two Sample T-test
potassium_min, mean (SD)                                        3.8 (0.6)      3.4 (0.6)   0.148  Two Sample T-test
inr_max, mean (SD)                                              1.5 (0.5)      1.4 (0.3)   0.594  Two Sample T-test
bun_max, mean (SD)                                            34.8 (30.4)    36.3 (22.5)   0.900  Two Sample T-test
wbc_max, mean (SD)                                            16.0 (10.9)    18.5 (13.5)   0.648  Two Sample T-test
bilirubin_min, mean (SD)                                        1.4 (2.3)      1.4 (1.6)   0.920  Two Sample T-test
calcium_total_min, mean (SD)                                    7.6 (0.6)      7.4 (1.1)   0.609  Two Sample T-test
calcium_ionized_min, mean (SD)                                  1.0 (0.1)      1.0 (0.2)   0.547  Two Sample T-test
ast_sgot_max, mean (SD)                                       67.6 (51.0)    69.0 (61.8)   0.957  Two Sample T-test
amylase_max, mean (SD)                                       87.5 (121.0)  112.5 (140.9)   0.670  Two Sample T-test
lipase_max, mean (SD)                                       160.5 (410.2)  107.5 (227.4)   0.716  Two Sample T-test
c_reactive_protein_max, mean (SD)                           134.5 (114.6)   86.4 (102.2)   0.322  Two Sample T-test
[1] Warning, Hartigan's Dip Test reports possible multimodal distributions for: bicarbonate_min, calcium_ionized_min, inr_max, sodium_max.
[2] Warning, Tukey test indicates far outliers in: platelet_min.
[3] Warning, test for normality reports non-normal distributions for: amylase_max, bilirubin_min, creatinine_max, lipase_max.

## Define useful functions

In [92]:
def run_kruskal(df_neg, df_pos):
    """Display results of a Kruskal-Wallis H test.
    
    @param df_neg: DataFrame containing the negative samples
    @param df_pos: DataFrame containing the positive samples
    """
    summary = pd.concat([df_neg.describe(), df_pos.describe()], axis=1)
    summary.columns = ['(-)', '(+)']
    display(summary)
    print (kruskal(df_neg, df_pos))

# Univariate association for non-normal distribution

> **TODO:** 
_demographics:_ age
_vitals:_ diasbp_min, heartrate_max, mingcs, resprate_max, spo2_min, sysbp_min, tempc_max, weight
_labs:_ amylase_max, bilirubin_min, creatinine_max, lipase_max

## 1. demographics: age

In [93]:
df_chl = df_chl.reset_index()
df_chl = df_chl.set_index(['subject_id','hadm_id'])
df_combined = df_demo.join(df_chl, how='inner')

In [94]:
kruskal(df_combined.query('chl_110 == 0').filter(['age']),
        df_combined.query('chl_110 == 1').filter(['age']))

KruskalResult(statistic=74.90376458296488, pvalue=4.942267490848014e-18)

## 2. vitals: diasbp_min, heartrate_max, mingcs, resprate_max, spo2_min, sysbp_min, tempc_max, weight

In [103]:
df_chl = df_chl.reset_index()
df_chl = df_chl.set_index(['subject_id','hadm_id','icustay_id'])
df_vitals = df_vitals.set_index(['subject_id','hadm_id','icustay_id'])
df_combined = df_vitals.join(df_chl, how='inner')

In [104]:
kruskal(df_combined.query('chl_110 == 0').filter(['diasbp_min']),
        df_combined.query('chl_110 == 1').filter(['diasbp_min']))

KruskalResult(statistic=38.689004722490154, pvalue=4.970008786477188e-10)

In [105]:
kruskal(df_combined.query('chl_110 == 0').filter(['heartrate_max']),
        df_combined.query('chl_110 == 1').filter(['heartrate_max']))

KruskalResult(statistic=22.91180898348445, pvalue=1.6960666298143548e-06)

In [106]:
kruskal(df_combined.query('chl_110 == 0').filter(['mingcs']),
        df_combined.query('chl_110 == 1').filter(['mingcs']))

KruskalResult(statistic=112.56148570813947, pvalue=2.691863077020637e-26)

In [107]:
kruskal(df_combined.query('chl_110 == 0').filter(['resprate_max']),
        df_combined.query('chl_110 == 1').filter(['resprate_max']))

KruskalResult(statistic=15.260401373878663, pvalue=9.365954504346782e-05)

In [108]:
kruskal(df_combined.query('chl_110 == 0').filter(['spo2_min']),
        df_combined.query('chl_110 == 1').filter(['spo2_min']))

KruskalResult(statistic=18.80065703582618, pvalue=1.4511712419459826e-05)

In [109]:
kruskal(df_combined.query('chl_110 == 0').filter(['sysbp_min']),
        df_combined.query('chl_110 == 1').filter(['sysbp_min']))

KruskalResult(statistic=34.87645872820526, pvalue=3.5130288958772705e-09)

In [110]:
kruskal(df_combined.query('chl_110 == 0').filter(['tempc_max']),
        df_combined.query('chl_110 == 1').filter(['tempc_max']))

KruskalResult(statistic=0.1417289714439337, pvalue=0.7065681949742861)

In [111]:
kruskal(df_combined.query('chl_110 == 0').filter(['weight']),
        df_combined.query('chl_110 == 1').filter(['weight']))

KruskalResult(statistic=76.10616495186768, pvalue=2.688167300987622e-18)

## 3. labs: amylase_max, bilirubin_min, creatinine_max, lipase_max

In [121]:
df_labs = df_labs.set_index(['subject_id','hadm_id','icustay_id'])
df_chl = df_chl.filter(['chl_110'])
df_combined = df_labs.join(df_chl, how='inner')

In [123]:
kruskal(df_combined.query('chl_110 == 0').filter(['amylase_max']),
        df_combined.query('chl_110 == 1').filter(['amylase_max']))

KruskalResult(statistic=1.1157024793388501, pvalue=0.29084570674353927)

In [124]:
kruskal(df_combined.query('chl_110 == 0').filter(['bilirubin_min']),
        df_combined.query('chl_110 == 1').filter(['bilirubin_min']))

KruskalResult(statistic=0.15400000000000583, pvalue=0.6947417515769789)

In [125]:
kruskal(df_combined.query('chl_110 == 0').filter(['creatinine_max']),
        df_combined.query('chl_110 == 1').filter(['creatinine_max']))

KruskalResult(statistic=0.4048072346501669, pvalue=0.5246169852940237)

In [126]:
kruskal(df_combined.query('chl_110 == 0').filter(['lipase_max']),
        df_combined.query('chl_110 == 1').filter(['lipase_max']))

KruskalResult(statistic=0.04465709728867311, pvalue=0.8326358146750303)